# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [42]:
#import libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from tensorflow import keras
from tensorflow.keras import layers

In [43]:
# your code here
game = pd.read_csv("tic-tac-toe.csv")
game.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [44]:
game.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB


In [45]:
game_cols = game.columns.values[game.dtypes == object]
game_dummies = pd.get_dummies(game, columns=game_cols, drop_first=True)
game_dummies.head()

,class,TL_o,TL_x,TM_o,TM_x,TR_o,TR_x,ML_o,ML_x,MM_o,MM_x,MR_o,MR_x,BL_o,BL_x,BM_o,BM_x,BR_o,BR_x
0,True,0,1,0,1,0,1,0,1,1,0,1,0,0,1,1,0,1,0
1,True,0,1,0,1,0,1,0,1,1,0,1,0,1,0,0,1,1,0
2,True,0,1,0,1,0,1,0,1,1,0,1,0,1,0,1,0,0,1
3,True,0,1,0,1,0,1,0,1,1,0,1,0,1,0,0,0,0,0
4,True,0,1,0,1,0,1,0,1,1,0,1,0,0,0,1,0,0,0


In [46]:
X = game.iloc[:, 0:9].values
y = game.iloc[:, 9:10].values

# Encode categorical variables as numeric
labelencoder_X = LabelEncoder()
for i in range(9):
    X[:, i] = labelencoder_X.fit_transform(X[:, i])
print(X[0])

# Encode target categorical variable
labelencoder_y = LabelEncoder()
y[:, 0] = labelencoder_y.fit_transform(y[:, 0])
print(y[0])

[2 2 2 2 1 1 2 1 1]
[ True]


In [47]:
enc = OneHotEncoder()
enc.fit(X)
X = enc.transform(X).toarray()

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [77]:
# your code here

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [91]:
#make model
def create_model(neurons=(100, 100), momentum=0.9):
    model = keras.Sequential()

    model.add(Dense(neurons[0], kernel_initializer='uniform', activation='relu', input_shape=(X_train.shape[1:]), use_bias=True))
    model.add(Dense(neurons[1], kernel_initializer='uniform', activation='relu', use_bias=True))
    model.add(Dense(7, kernel_initializer='uniform', activation='sigmoid', use_bias=True))

    model.compile(loss='sparse_categorical_crossentropy', optimizer= 'adam',metrics=['accuracy'] )

    return model

In [92]:
#create model
nnet = create_model()
nnet.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_45 (Dense)             (None, 100)               2800      
_________________________________________________________________
dense_46 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_47 (Dense)             (None, 7)                 707       
Total params: 13,607
Trainable params: 13,607
Non-trainable params: 0
_________________________________________________________________


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [93]:
# your code here
history_callback = nnet.fit(X_train, y_train, batch_size=10, epochs=100, verbose=1, validation_data=(X_test, y_test))

Epoch 1/100
77/77 [==============================] - 0s 3ms/step - loss: 1.1430 - accuracy: 0.6475 - val_loss: 0.6875 - val_accuracy: 0.6667
Epoch 2/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6545 - accuracy: 0.6501 - val_loss: 0.6340 - val_accuracy: 0.6667
Epoch 3/100
77/77 [==============================] - 0s 2ms/step - loss: 0.6309 - accuracy: 0.6501 - val_loss: 0.5936 - val_accuracy: 0.6667
Epoch 4/100
77/77 [==============================] - 0s 2ms/step - loss: 0.5896 - accuracy: 0.6501 - val_loss: 0.5472 - val_accuracy: 0.6667
Epoch 5/100
77/77 [==============================] - 0s 2ms/step - loss: 0.5526 - accuracy: 0.6501 - val_loss: 0.5252 - val_accuracy: 0.6667
Epoch 6/100
77/77 [==============================] - 0s 2ms/step - loss: 0.5227 - accuracy: 0.6501 - val_loss: 0.5166 - val_accuracy: 0.6667
Epoch 7/100
77/77 [==============================] - 0s 2ms/step - loss: 0.5023 - accuracy: 0.6501 - val_loss: 0.4967 - val_accuracy: 0.6667
Epoch 8/100
7

Epoch 59/100
77/77 [==============================] - 0s 2ms/step - loss: 8.5853e-04 - accuracy: 1.0000 - val_loss: 0.0418 - val_accuracy: 0.9844
Epoch 60/100
77/77 [==============================] - 0s 2ms/step - loss: 7.9541e-04 - accuracy: 1.0000 - val_loss: 0.0395 - val_accuracy: 0.9844
Epoch 61/100
77/77 [==============================] - 0s 2ms/step - loss: 7.2285e-04 - accuracy: 1.0000 - val_loss: 0.0424 - val_accuracy: 0.9844
Epoch 62/100
77/77 [==============================] - 0s 2ms/step - loss: 6.2586e-04 - accuracy: 1.0000 - val_loss: 0.0405 - val_accuracy: 0.9844
Epoch 63/100
77/77 [==============================] - 0s 2ms/step - loss: 6.1877e-04 - accuracy: 1.0000 - val_loss: 0.0372 - val_accuracy: 0.9896
Epoch 64/100
77/77 [==============================] - 0s 2ms/step - loss: 6.0079e-04 - accuracy: 1.0000 - val_loss: 0.0439 - val_accuracy: 0.9896
Epoch 65/100
77/77 [==============================] - 0s 2ms/step - loss: 5.6317e-04 - accuracy: 1.0000 - val_loss: 0.0424 -

In [94]:
score = nnet.evaluate(X_test, y_test, verbose=1)
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])

# Predicting the test set results
y_pred = nnet.predict_classes(X_test)

6/6 [==============================] - 0s 1ms/step - loss: 0.0464 - accuracy: 0.9896


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# your code here : No need as my accuracy rate is really high with less loss

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here : learning rate can be quite helpful